In [34]:
from pathlib import Path
from tempfile import TemporaryDirectory

import torch
from loguru import logger
from mido import MidiFile

from yoshimidi import player
from yoshimidi.data.parse import midi_parsing, one_hot_parsing, token_parsing, track_parsing

In [146]:
PATH_INDEX = 10
midi_path = list(Path("../out/dataset/01_raw/lmd_full/").rglob("*.mid"))[PATH_INDEX]

In [147]:
player.play(midi_path)

2023-08-26 19:33:45.159 | INFO     | yoshimidi.player:play:12 - Playing ../out/dataset/01_raw/lmd_full/9/952df587b9fff5adbeb5f40b288fa64b.mid
2023-08-26 19:33:56.611 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [148]:
CHANNEL_INDEX = 1
midi_file = MidiFile(midi_path)
channels = set(msg.channel for msg in midi_file if hasattr(msg, "channel"))
logger.info("num channels: {}".format(len(channels)))
ym_track = track_parsing.from_midi(midi_file)
assert ym_track is not None
ym_tokens = [token_parsing.from_channel(channel) for channel in ym_track.channels.values()]
ym_tokens = [ym_tokens[CHANNEL_INDEX]]
ym_one_hot = one_hot_parsing.from_tokens(ym_tokens[0], device=torch.device("cpu"), dtype=torch.float32)

ym_track_recons = track_parsing.from_tokens(ym_tokens)
midi_file_recons = midi_parsing.from_tracks([ym_track_recons])

with TemporaryDirectory() as temp:
    path = Path(temp) / "recons.mid"
    midi_file_recons.save(path)
    player.play(path)

2023-08-26 19:33:57.896 | INFO     | __main__:<module>:4 - num channels: 16
2023-08-26 19:33:58.022 | INFO     | yoshimidi.player:play:12 - Playing /var/folders/7w/66fh_d3s5hb0br9f7wtqww1h0000gn/T/tmp18nvn28g/recons.mid
2023-08-26 19:34:13.061 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt
